In [22]:
import sys
sys.path.insert(1, r"C:/Users/micha/Documents/Imperial Courses/Thesis/IAEA-thesis")
from Stage1.GAT.GATModel import GraphAttentionNetwork
from Single_Website_Download.Download import main
import torch
from Stage1.ExtractingGraphs.HTMLtoGraph import html_to_graph, EdgeFeatures
from Stage1.ExtractingGraphs.seleniumDriver import *
from Stage1.ExtractingLabels.swde_label_extraction import label_extraction, _find_matches
from Stage1.tree_helpers import *
from pathlib import Path
from scipy import sparse
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

In [23]:
#Import model
model = GraphAttentionNetwork(in_dim = 114, edge_in_dim = 200, edge_emb_dim = 32, hidden1 = 32, hidden2 = 32, hidden3 = 8, heads = 2)
state_dict = torch.load("../../Stage1/GAT/FULLTRAINEDALLDATAModelf1-75-learning.pt", map_location=torch.device(device))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [24]:
#url = "C:\\Users\\micha\\Documents\\Imperial Courses\\Thesis\\IAEA-thesis\\data\\swde\\sourceCode\\sourceCode\\movie\\movie\\movie-allmovie(2000)\\0000.htm"
url = r"https://www.nucnet.org/news/parliament-resolution-paves-way-for-establishing-nuclear-energy-legislation-6-4-2024"
htmlFile = Path("C:/Users/micha/Documents/Imperial Courses/Thesis/IAEA-thesis/data/websites/test.html")
main(url,1,htmlFile)

Saved https://www.nucnet.org/news/parliament-resolution-paves-way-for-establishing-nuclear-energy-legislation-6-4-2024 → C:\Users\micha\Documents\Imperial Courses\Thesis\IAEA-thesis\data\websites\test.html
Page limit hit


In [25]:
driver_init(True)
restart_Driver(True)
_, X_npy, E_sparse, edge_index, bbox = html_to_graph(htmlFile, get_Driver())
quit_driver()
X_sparse = sparse.csr_matrix(X_npy)
E_sparse = sparse.csr_matrix(E_sparse)

In [26]:
def get_random_candidate_edges(filepath, X_npy):
    TXTLENGTH = 400
    tree = load_html_as_tree(filepath)
    node2index, index2node = bfs_index_map(tree)
    txtNodes = []
    coords = []

    stack = [tree.getroot()]
    while stack:
        #limit to nodes with text less than a certain length. once that node has been read, do not grab its children
        node = stack.pop()
        txtlen = len(get_node_text(node))

        if txtlen > 2 and txtlen < TXTLENGTH:
            txtNodes.append(node2index[node])
            continue  # skip this node's children entirely

        # Reverse so we process in document order.
        for child in reversed(list(node)):
            stack.append(child)

    #This logic should make an undirected graph
    for i in txtNodes:
        for j in txtNodes:
            if i != j:
                coords.append((i,j))

    XPaths: Dict[etree._Element, str] = {}
    for el in node2index:
        xp = tree.getpath(el)
        XPaths[el] = xp
    parentMap, depthMap = build_parent_and_depth_maps(tree)
    
    features = []
    for i, j in coords:
        features.append(EdgeFeatures(
            edgeStart=i,
            edgeEnd=j,
            edgeStartNode=index2node[i],
            edgeEndNode=index2node[j],
            X=X_npy,
            bboxs=bbox,
            parentMap=parentMap,
            depthMap=depthMap,
            XPaths=XPaths
        ))

    return np.array(coords), features

def get_specific_candidate_edges(filepath, txt, X_npy):
    TXTLENGTH = 400
    tree = load_html_as_tree(filepath)
    node2index, index2node = bfs_index_map(tree)
    parentMap, depthMap = build_parent_and_depth_maps(tree)
    node = _find_matches(tree, txt, depthMap)[0]
    nodeindex = node2index[node]
    
    coords = []

    #This logic should make an undirected graph
    for n, _ in iter_elements_with_direct_text(tree):
        i = node2index[n]
        if i != nodeindex:
            coords.append((i,nodeindex))
            coords.append((nodeindex, i))

    XPaths: Dict[etree._Element, str] = {}
    for el in node2index:
        xp = tree.getpath(el)
        XPaths[el] = xp
    
    features = []
    for i, j in coords:
        features.append(EdgeFeatures(
            edgeStart=i,
            edgeEnd=j,
            edgeStartNode=index2node[i],
            edgeEndNode=index2node[j],
            X=X_npy,
            bboxs=bbox,
            parentMap=parentMap,
            depthMap=depthMap,
            XPaths=XPaths
        ))

    return np.array(coords), features

In [44]:
label_index, label_features = get_random_candidate_edges(htmlFile, X_npy)
label_index = np.array(label_index)
label_features = sparse.csr_matrix(label_features)

In [45]:
def _npz_to_csr(csr: sparse.csr, dtype=torch.float32):
    crow = torch.from_numpy(csr.indptr.astype(np.int64))
    col  = torch.from_numpy(csr.indices.astype(np.int64))
    val  = torch.from_numpy(csr.data).to(dtype)
    return torch.sparse_csr_tensor(
        crow, col, val, size=csr.shape, dtype=dtype, requires_grad=False
    )

def _npy_to_tensor(arr: np.ndarray, dtype=torch.long):
    return torch.from_numpy(arr).to(dtype).t().contiguous()

X, Aei, Aef, Lei, Lef = _npz_to_csr(X_sparse), _npy_to_tensor(edge_index), _npz_to_csr(E_sparse), _npy_to_tensor(label_index), _npz_to_csr(label_features)
X, Aei, Aef, Lei, Lef = X.to(device), Aei.to(device), Aef.to(device), Lei.to(device), Lef.to(device)
print(Lef.shape)

torch.Size([1482, 200])


In [46]:
model.eval()
logits = model(X, Aei, Aef, Lei, Lef)
probs  = torch.sigmoid(logits)
probs.shape

torch.Size([1482])

In [47]:
order = np.argsort(probs.squeeze().tolist())[::-1]
sorted_label_index = label_index[order]
sorted_label_index

array([[ 77,  78],
       [139,  45],
       [140,  45],
       ...,
       [187,  36],
       [186,  36],
       [186,  35]], shape=(1482, 2))

In [48]:
tree = load_html_as_tree(htmlFile)
_, index2node = bfs_index_map(tree)

for label in sorted_label_index:
    print(get_node_text(index2node[label[0]], True).strip(), "->", get_node_text(index2node[label[1]], True).strip())
    print(tree.getpath(index2node[label[0]]), "->", tree.getpath(index2node[label[1]]))

tags -> estoniaparliamentresolutionlawregulator
/html/body/div/div[2]/section[1]/div/div[2]/h4 -> /html/body/div/div[2]/section[1]/div/div[2]/ul
newbuildestoniaclimatestrategyincludesplansfouruptofoursmallmodularreactors12february2024 -> related
/html/body/div/div[2]/section[1]/div/div[3]/div[1]/div[2] -> /html/body/div/div[2]/section[1]/div/h4
smallmodularreactorsestoniafermienergiamovesbacktimelineforfirstoperationalsmrto20357february2024 -> related
/html/body/div/div[2]/section[1]/div/div[3]/div[1]/div[3] -> /html/body/div/div[2]/section[1]/div/h4
estoniaparliamentresolutionlawregulator -> tags
/html/body/div/div[2]/section[1]/div/div[2]/ul -> /html/body/div/div[2]/section[1]/div/div[2]/h4
nuclearpoliticsestoniadebateonnuclearprogrammeprogressesincountrysparliament29april2024 -> related
/html/body/div/div[2]/section[1]/div/div[3]/div[1]/div[1] -> /html/body/div/div[2]/section[1]/div/h4
related -> newbuildestoniaclimatestrategyincludesplansfouruptofoursmallmodularreactors12february20

1) see how many external links I am picking up, if its an acceptable amount, dw about it
filter out links in html that point to external webpages

2) compare this to just putting it all into an llm. make this a prompting model